In [ ]:
import gensim
from gensim.models import FastText

import pandas as pd
import numpy as np
import random


## Preparing dataset for FastText vectorization
------------------------

In [ ]:
fpath = '/content/ru_syn_dataset_mod1.json'

In [ ]:
df = pd.read_json(fpath)

In [ ]:
df

,word,synonyms,synonyms_count,word_clear,synonyms_clear
0,10-разрядный,[десятиразрядный],1,10-разрядный,[десятиразрядный]
1,12-разрядный,[двенадцатиразрядный],1,12-разрядный,[двенадцатиразрядный]
2,128-разрядный,[стодвадцативосьмиразрядный],1,128-разрядный,[стодвадцативосьмиразрядный]
3,14-разрядный,[четырнадцатиразрядный],1,14-разрядный,[четырнадцатиразрядный]
4,16-разрядный,[шестнадцатиразрядный],1,16-разрядный,[шестнадцатиразрядный]
...,...,...,...,...,...
337355,ётёх,"[развалины, брошенный дом]",2,ётёх,"[развалины, брошенный-дом]"
337356,ёухикко,[инструмент],1,ёухикко,[инструмент]
337357,ёфикатор,"[ёделя, ёдельник]",2,ёфикатор,"[ёделя, ёдельник]"
337358,ёхнувший,[сдвинувший],1,ёхнувший,[сдвинувший]


In [ ]:
sentences = []
for index, row in df.iterrows():
    for syn in row['synonyms_clear']:
        sentences.append([row['word_clear'], syn])

In [ ]:
df_for_ft = pd.DataFrame(data=sentences, columns=['word1', 'word2'])

In [ ]:
df_for_ft.to_json('/content/ru_syn_dataset_modft.json')

## Reading dataset (df_fot_ft)
----

In [ ]:
df = pd.read_json('/content/ru_syn_dataset_modft.json')

In [ ]:
df

,word1,word2
0,10-разрядный,десятиразрядный
1,12-разрядный,двенадцатиразрядный
2,128-разрядный,стодвадцативосьмиразрядный
3,14-разрядный,четырнадцатиразрядный
4,16-разрядный,шестнадцатиразрядный
...,...,...
3826501,ёухикко,инструмент
3826502,ёфикатор,ёделя
3826503,ёфикатор,ёдельник
3826504,ёхнувший,сдвинувший


In [ ]:
sentences = [[row['word1'], row['word2']] for _, row in df.iterrows()]

## FastText model training
----------


In [ ]:
model = FastText(sentences=None,
                 vector_size=150,
                 window=2,
                 min_count=1,
                 workers=15,
                 sg=1,
                 hs=1,
                 negative=0,
                 min_n=2,
                 max_n=6)

In [ ]:
random.seed(42)
random.shuffle(sentences)

In [ ]:
model.build_vocab(sentences, progress_per=100000)

In [ ]:
%%time

model.train(sentences,
            total_examples=len(sentences),
            epochs=50,
            report_delay=10.0)

CPU times: user 3h 13min 30s, sys: 1min 16s, total: 3h 14min 46s
Wall time: 2h 54min 31s


(382650600, 382650600)

## Testing model
-----

In [ ]:
model.wv.most_similar('перспектива', topn=10)

[('перспективы', 0.8648058772087097),
 ('европерспектива', 0.732286274433136),
 ('перспективность', 0.6640074849128723),
 ('перспективно', 0.652469277381897),
 ('перспективная-оценка', 0.6476495862007141),
 ('перспекс', 0.6435973048210144),
 ('перспективный', 0.6313502192497253),
 ('в-перспективе', 0.6226367354393005),
 ('перспективнейший', 0.614008903503418),
 ('среднесрочная-перспектива', 0.6020455956459045)]

In [ ]:
model.save('/content/synonyms_ft.model2')

In [ ]:
words = ['деньги', 'задачки', 'деньжата', 'карьера', 'новые-знания', 'новые-навыки', 'система-обучения', 'коллеги', 'наставник', 'зарплата', 'задачи', 'область-работы',
         'различные-проекты', 'карьерный-рост', 'обстановка', 'атмосфера', 'коллектив', 'мероприятия', 'корпоративы',
        'зп', 'зарплата', 'деньги', 'комфорт', 'статус', 'статусность', 'престиж', 'развитие', 'курсы', 'обучение', 'интересные-задачи', 'профессиональный-рост',
         'обучение', 'карьера', 'карьерный-рост', 'перспективы', 'стабильность']

In [ ]:
model.wv.rank_by_centrality(['задачки', 'задачи', 'интересные-задачи'])

[(0.87133694, 'задачи'),
 (0.8077354, 'задачки'),
 (0.77999717, 'интересные-задачи')]

In [ ]:
vec_list = [model.wv[w] for w in words]

In [ ]:
import sklearn
from sklearn.cluster import DBSCAN
import numpy as np

In [ ]:
vec_list_ = np.array(vec_list)

In [ ]:
clustering = DBSCAN(eps=0.5, min_samples=2, metric='cosine', algorithm='auto').fit(vec_list_)

In [ ]:
labels = clustering.labels_

In [ ]:
words_np = np.array(words)

In [ ]:
words_np[np.where(labels==0)[0]]

array(['деньги', 'деньжата', 'деньги'], dtype='<U21')

In [ ]:
cloud = []
for label in np.unique(labels):
    cloud.append(words_np[np.where(labels == label)[0]])

In [ ]:
cloud

[array(['наставник', 'область-работы', 'различные-проекты', 'атмосфера',
        'мероприятия', 'корпоративы', 'зп', 'комфорт', 'курсы',
        'перспективы'], dtype='<U21'),
 array(['деньги', 'деньжата', 'деньги'], dtype='<U21'),
 array(['задачки', 'задачи', 'интересные-задачи'], dtype='<U21'),
 array(['карьера', 'карьера'], dtype='<U21'),
 array(['новые-знания', 'новые-навыки'], dtype='<U21'),
 array(['система-обучения', 'обстановка', 'статус', 'статусность',
        'престиж', 'развитие', 'обучение', 'обучение', 'стабильность'],
       dtype='<U21'),
 array(['коллеги', 'коллектив'], dtype='<U21'),
 array(['зарплата', 'зарплата'], dtype='<U21'),
 array(['карьерный-рост', 'профессиональный-рост', 'карьерный-рост'],
       dtype='<U21')]